# Comparativo: BFS vs Costo Uniforme (Best-First) vs Profundidad Iterativa (IDS)

Este notebook compara **tres algoritmos de búsqueda no informada** usando el mismo problema clásico del **mapa de Rumania** (Arad → Bucharest):

1. **BFS (Breadth-First Search / Búsqueda en Anchura)**
2. **Costo Uniforme (Uniform-Cost Search)** implementado como **Best-First Search** con `f(n)=g(n)`
3. **IDS (Iterative Deepening Search / Búsqueda en Profundidad Iterativa)**

El objetivo es ver, con el mismo grafo:
- el **camino** encontrado
- el **número de pasos** (aristas)
- el **costo total** (solo aplica cuando usamos costos)



## 0) Importaciones

In [ ]:
import heapq
from collections import deque

## 1) Definir el problema (grafo de Rumania)

Usamos:
- `ACTIONS[ciudad] -> lista de vecinas`
- `COSTS[(ciudad, vecina)] -> distancia`

Nota: BFS e IDS ignoran costos; Costo Uniforme sí los usa.


In [ ]:
ACTIONS = {
    "Arad": ["Sibiu", "Timisoara", "Zerind"],
    "Zerind": ["Arad", "Oradea"],
    "Oradea": ["Zerind", "Sibiu"],
    "Sibiu": ["Arad", "Oradea", "Fagaras", "Rimnicu Vilcea"],
    "Timisoara": ["Arad", "Lugoj"],
    "Lugoj": ["Timisoara", "Mehadia"],
    "Mehadia": ["Lugoj", "Drobeta"],
    "Drobeta": ["Mehadia", "Craiova"],
    "Craiova": ["Drobeta", "Rimnicu Vilcea", "Pitesti"],
    "Rimnicu Vilcea": ["Sibiu", "Craiova", "Pitesti"],
    "Fagaras": ["Sibiu", "Bucharest"],
    "Pitesti": ["Rimnicu Vilcea", "Craiova", "Bucharest"],
    "Bucharest": ["Fagaras", "Pitesti"]
}

COSTS = {
    ("Arad", "Sibiu"): 140, ("Sibiu", "Arad"): 140,
    ("Arad", "Timisoara"): 118, ("Timisoara", "Arad"): 118,
    ("Arad", "Zerind"): 75, ("Zerind", "Arad"): 75,
    ("Zerind", "Oradea"): 71, ("Oradea", "Zerind"): 71,
    ("Oradea", "Sibiu"): 151, ("Sibiu", "Oradea"): 151,
    ("Sibiu", "Fagaras"): 99, ("Fagaras", "Sibiu"): 99,
    ("Sibiu", "Rimnicu Vilcea"): 80, ("Rimnicu Vilcea", "Sibiu"): 80,
    ("Timisoara", "Lugoj"): 111, ("Lugoj", "Timisoara"): 111,
    ("Lugoj", "Mehadia"): 70, ("Mehadia", "Lugoj"): 70,
    ("Mehadia", "Drobeta"): 75, ("Drobeta", "Mehadia"): 75,
    ("Drobeta", "Craiova"): 120, ("Craiova", "Drobeta"): 120,
    ("Craiova", "Rimnicu Vilcea"): 146, ("Rimnicu Vilcea", "Craiova"): 146,
    ("Craiova", "Pitesti"): 138, ("Pitesti", "Craiova"): 138,
    ("Rimnicu Vilcea", "Pitesti"): 97, ("Pitesti", "Rimnicu Vilcea"): 97,
    ("Fagaras", "Bucharest"): 211, ("Bucharest", "Fagaras"): 211,
    ("Pitesti", "Bucharest"): 101, ("Bucharest", "Pitesti"): 101
}

def actions_fn(s):
    return ACTIONS.get(s, [])

def result_fn(s, a):
    # en este encoding, la acción es "ir a la ciudad a"
    return a

def action_cost_fn(s, a, s2):
    return COSTS.get((s, a), float("inf"))


## 2) Utilidades comunes

Funciones para reconstrucción de camino, conteo de pasos y cálculo de costo total.


In [ ]:
def reconstruct_path(goal_node):
    if goal_node is None:
        return None
    path = []
    n = goal_node
    while n is not None:
        path.append(n.state)
        n = n.parent
    return list(reversed(path))

def steps_of(path):
    return None if path is None else max(0, len(path) - 1)

def total_cost_of(path):
    if path is None:
        return None
    if len(path) <= 1:
        return 0
    cost = 0
    for u, v in zip(path[:-1], path[1:]):
        cost += COSTS.get((u, v), float("inf"))
    return cost


## 3) BFS (Búsqueda en Anchura)

**Idea:** frontera como **cola FIFO**; explora por niveles.

- Óptima en número de pasos cuando los costos son uniformes.
- No garantiza menor costo cuando las aristas tienen distancias distintas.


In [ ]:
class NodeBFS:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action

def expand_bfs(node):
    for a in actions_fn(node.state):
        s2 = result_fn(node.state, a)
        yield NodeBFS(state=s2, parent=node, action=a)

def bfs(initial, goal):
    root = NodeBFS(initial)
    if root.state == goal:
        return root

    frontier = deque([root])
    reached = {initial}

    while frontier:
        node = frontier.popleft()
        for child in expand_bfs(node):
            if child.state not in reached:
                if child.state == goal:
                    return child
                reached.add(child.state)
                frontier.append(child)
    return None


## 4) Costo Uniforme (Best-First con f(n)=g(n))

**Idea:** frontera como **cola de prioridad** ordenada por `g(n)` (costo acumulado).

- Óptimo en costo total si todos los costos son no negativos.
- Puede explorar más nodos que BFS en términos de pasos, pero encuentra el camino más barato.


In [ ]:
class NodeUCS:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __lt__(self, other):
        return self.path_cost < other.path_cost

def expand_ucs(node):
    for a in actions_fn(node.state):
        s2 = result_fn(node.state, a)
        new_cost = node.path_cost + action_cost_fn(node.state, a, s2)
        yield NodeUCS(state=s2, parent=node, action=a, path_cost=new_cost)

def ucs(initial, goal):
    root = NodeUCS(initial)

    frontier = [(root.path_cost, root)]
    heapq.heapify(frontier)

    reached = {initial: root}  # mejor nodo por estado

    while frontier:
        _, node = heapq.heappop(frontier)

        if node.state == goal:
            return node

        for child in expand_ucs(node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                heapq.heappush(frontier, (child.path_cost, child))

    return None


## 5) IDS (Búsqueda en Profundidad Iterativa)

**Idea:** ejecutar una DFS con límite (DLS) para límites 0,1,2,… hasta encontrar solución.

- Óptima en número de pasos (con costos uniformes).
- Mucho menor memoria que BFS, pero repite trabajo.


In [ ]:
class NodeIDS:
    def __init__(self, state, parent=None, action=None, depth=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth

def expand_ids(node):
    children = []
    for a in actions_fn(node.state):
        s2 = result_fn(node.state, a)
        children.append(NodeIDS(state=s2, parent=node, action=a, depth=node.depth + 1))
    return children

def is_cycle_ids(node):
    cur = node.parent
    while cur is not None:
        if cur.state == node.state:
            return True
        cur = cur.parent
    return False

def dls(initial, goal, limit):
    frontier = deque([NodeIDS(initial)])  # pila LIFO
    result = "failure"

    while frontier:
        node = frontier.pop()

        if node.state == goal:
            return node

        if node.depth > limit:
            result = "cutoff"
        else:
            for child in expand_ids(node):
                if not is_cycle_ids(child):
                    frontier.append(child)

    return result

def ids(initial, goal, max_depth=50):
    for limit in range(max_depth):
        r = dls(initial, goal, limit)
        if r != "cutoff":
            return r  # solución (NodeIDS) o failure
    return None


## 6) Ejecutar los tres algoritmos (Arad → Bucharest)

In [ ]:
initial = "Arad"
goal = "Bucharest"

# BFS
bfs_goal = bfs(initial, goal)
bfs_path = reconstruct_path(bfs_goal)

# UCS / Costo Uniforme
ucs_goal = ucs(initial, goal)
ucs_path = reconstruct_path(ucs_goal)

# IDS
ids_goal = ids(initial, goal, max_depth=50)
ids_path = reconstruct_path(ids_goal) if isinstance(ids_goal, NodeIDS) else None

bfs_path, ucs_path, ids_path


## 7) Resumen de resultados

Interpretación rápida:
- **BFS / IDS**: minimizan pasos (si el problema se modela con costos uniformes).
- **Costo uniforme**: minimiza el **costo total** (distancia).


In [ ]:
results = [
    {
        "Algoritmo": "BFS (Anchura)",
        "Camino": bfs_path,
        "Pasos": steps_of(bfs_path),
        "Costo total": total_cost_of(bfs_path),
        "Criterio de optimalidad": "Mínimos pasos (si costos uniformes)"
    },
    {
        "Algoritmo": "Costo uniforme (Best-First con f=g)",
        "Camino": ucs_path,
        "Pasos": steps_of(ucs_path),
        "Costo total": total_cost_of(ucs_path),
        "Criterio de optimalidad": "Mínimo costo total"
    },
    {
        "Algoritmo": "IDS (Profundidad iterativa)",
        "Camino": ids_path,
        "Pasos": steps_of(ids_path),
        "Costo total": total_cost_of(ids_path),
        "Criterio de optimalidad": "Mínimos pasos (si costos uniformes)"
    }
]

for r in results:
    print("—" * 80)
    print(r["Algoritmo"])
    print("Camino:", r["Camino"])
    print("Pasos:", r["Pasos"])
    print("Costo total:", r["Costo total"])
    print("Optimalidad:", r["Criterio de optimalidad"])


## 8) Tabla comparativa de propiedades (conceptual)


| Algoritmo | Completo | Óptimo | Tiempo (peor caso) | Memoria (peor caso) |
|---|---|---|---|---|
| BFS | Sí (si factor de ramificación finito) | Sí en pasos (costos uniformes) | Exponencial | Exponencial |
| Costo uniforme | Sí (costos > 0) | Sí en costo | Puede ser grande (depende de costos) | Puede ser grande |
| IDS | Sí (si existe solución y límite crece) | Sí en pasos (costos uniformes) | Exponencial (con repetición) | Lineal en profundidad |

Notas:
- La complejidad exacta depende del **factor de ramificación b**, la **profundidad de la solución d** y el **límite máximo**.
- En el mapa de Rumania (pequeño), las diferencias se ven mejor en los caminos/costos obtenidos.
